In [ ]:
import pandas as pd
import numpy as np
from sklearn import cross_validation, metrics
from sklearn.ensemble import BaggingClassifier
from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
%matplotlib inline

df = pd.read_csv('datasets/workproject')
                 
df = df.dropna()

listprice = []
for i in df.rent:
    if i< 20000:
        listprice.append("cheap")
    elif i > 20000 and i < 45000:
        listprice.append("medium")
    elif i > 45000 and i < 80000:
        listprice.append("high")
    else:
        listprice.append("very high")

df["rentyp"] = listprice

columns_to_use = [ "gross", "salar", "habit", "bathh", "edad", "lat", "lng", "type", "distance", "nivel", "district_label" ]


X = df[columns_to_use]

y = df.rentyp

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
                 
                 
baggingknn = BaggingClassifier(KNeighborsClassifier())

bagging_params = {'n_estimators': [10, 20],
                  'max_samples': [0.7, 1.0],
                  'max_features': [0.7, 1.0],
                  'bootstrap_features': [True, False]}


gsbaggingknn = GridSearchCV(baggingknn,
                            bagging_params, n_jobs=-1,
                            cv=KFold(len(y_train), n_folds=3, shuffle=True))

model = gsbaggingknn.fit(X_train, y_train)

filename = "Models/Range/rent_predict_range.joblib.pkl"

_ = joblib.dump(model, filename, compress=9)
